In [1]:
# Create CNN using numpy

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import cv2
import random
import pickle

# Load data
import keras.datasets.mnist as mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Normalize data
X_train = X_train/255.0
X_test = X_test/255.0

# Reshape data
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

# One-hot encoding
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


2022-12-30 20:30:33.415088: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-30 20:30:33.551594: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-30 20:30:34.237915: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/venom/miniconda3/lib/python3.8/site-packages/cv2/../../lib64:/home/venom/lib/:/us

In [2]:
# Crewate CNN using numpy
class CNN:
    def __init__(self, X_train, y_train, X_test, y_test, epochs, learning_rate, batch_size):
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.loss = []
        self.accuracy = []
        self.val_loss = []
        self.val_accuracy = []
        self.history = {'loss': self.loss, 'accuracy': self.accuracy, 'val_loss': self.val_loss, 'val_accuracy': self.val_accuracy}
        self.W1 = np.random.randn(3, 3, 1, 32) * np.sqrt(2/(3*3*1))
        self.b1 = np.zeros((1, 1, 1, 32))
        self.W2 = np.random.randn(3, 3, 32, 64) * np.sqrt(2/(3*3*32))
        self.b2 = np.zeros((1, 1, 1, 64))
        self.W3 = np.random.randn(7*7*64, 128) * np.sqrt(2/(7*7*64))
        self.b3 = np.zeros((1, 128))
        self.W4 = np.random.randn(128, 10) * np.sqrt(2/(128))
        self.b4 = np.zeros((1, 10))
        self.params = {'W1': self.W1, 'b1': self.b1, 'W2': self.W2, 'b2': self.b2, 'W3': self.W3, 'b3': self.b3, 'W4': self.W4, 'b4': self.b4}
        
    def batchedConv2d(self, X, W, b, stride, padding):
        (m, n_H_prev, n_W_prev, n_C_prev) = X.shape
        (f, f, n_C_prev, n_C) = W.shape
        n_H = int((n_H_prev - f)/stride) + 1
        n_W = int((n_W_prev - f)/stride) + 1
        Z = np.zeros((m, n_H, n_W, n_C))
        for h in range(n_H):
            for w in range(n_W):
                for c in range(n_C):
                    vert_start = h*stride
                    vert_end = vert_start + f
                    horiz_start = w*stride
                    horiz_end = horiz_start + f
                    a_slice_prev = X[:, vert_start:vert_end, horiz_start:horiz_end, :]
                    Z[:, h, w, c] = np.sum(a_slice_prev*W[:, :, :, c], axis=(1, 2, 3)) + b[:, :, :, c]
        return Z

    def relu(self, Z):
        A = np.maximum(0, Z)
        return A
    
    def maxpool2d(self, A, f, stride):
        (n_H_prev, n_W_prev, n_C_prev) = A.shape
        n_H = int((n_H_prev - f)/stride) + 1
        n_W = int((n_W_prev - f)/stride) + 1
        n_C = n_C_prev
        A = A.reshape(n_H_prev, n_W_prev, n_C_prev)
        Z = np.zeros((n_H, n_W, n_C))
        for h in range(n_H):
            for w in range(n_W):
                for c in range(n_C):
                    vert_start = h*stride
                    vert_end = vert_start + f
                    horiz_start = w*stride
                    horiz_end = horiz_start + f
                    a_prev_slice = A[vert_start:vert_end, horiz_start:horiz_end, c]
                    Z[h, w, c] = np.max(a_prev_slice)
        return Z

    def flatten(self, A):
        (n_H, n_W, n_C) = A.shape
        A = A.reshape(n_H*n_W*n_C, 1)
        return A

    def softmax(self, Z):
        A = np.exp(Z)/np.sum(np.exp(Z), axis=1, keepdims=True)
        return A
    
    def forward_propagation(self, X):
        Z1 = self.conv2d(X, self.W1, self.b1, 1, 1)
        A1 = self.relu(Z1)
        P1 = self.maxpool2d(A1, 2, 2)
        Z2 = self.conv2d(P1, self.W2, self.b2, 1, 1)
        A2 = self.relu(Z2)
        P2 = self.maxpool2d(A2, 2, 2)
        F = self.flatten(P2)
        Z3 = F.dot(self.W3) + self.b3
        A3 = self.relu(Z3)
        Z4 = A3.dot(self.W4) + self.b4
        A4 = self.softmax(Z4)
        return A4
    
    def compute_cost(self, A4, y):
        m = y.shape[0]
        cost = -np.sum(y*np.log(A4))/m
        return cost
    
    def conv2d_backward(self, dZ, X, W, b, stride, padding):
        (n_H_prev, n_W_prev, n_C_prev) = X.shape
        (f, f, n_C_prev, n_C) = W.shape
        (n_H, n_W, n_C) = dZ.shape
        dW = np.zeros((f, f, n_C_prev, n_C))
        db = np.zeros((1, 1, 1, n_C))
        dX = np.zeros((n_H_prev, n_W_prev, n_C_prev))
        X_pad = np.pad(X, ((0, 0), (padding, padding), (padding, padding), (0, 0)), 'constant')
        dX_pad = np.pad(dX, ((0, 0), (padding, padding), (padding, padding), (0, 0)), 'constant')
        for h in range(n_H):
            for w in range(n_W):
                for c in range(n_C):
                    vert_start = h*stride
                    vert_end = vert_start + f
                    horiz_start = w*stride
                    horiz_end = horiz_start + f
                    a_slice = X_pad[:, vert_start:vert_end, horiz_start:horiz_end, :]
                    dX_pad[:, vert_start:vert_end, horiz_start:horiz_end, :] += W[:, :, :, c] * dZ[h, w, c]
                    dW[:, :, :, c] += a_slice * dZ[h, w, c]
                    db[:, :, :, c] += dZ[h, w, c]
        dX = dX_pad[:, padding:-padding, padding:-padding, :]
        return dX, dW, db
    
    def relu_backward(self, dA, Z):
        dZ = np.array(dA, copy=True)
        dZ[Z <= 0] = 0
        return dZ
    
    def maxpool2d_backward(self, dA, A, f, stride):
        (n_H_prev, n_W_prev, n_C_prev) = A.shape
        (n_H, n_W, n_C) = dA.shape
        dA = dA.reshape(n_H, n_W, n_C)
        dX = np.zeros((n_H_prev, n_W_prev, n_C_prev))
        for h in range(n_H):
            for w in range(n_W):
                for c in range(n_C):
                    vert_start = h*stride
                    vert_end = vert_start + f
                    horiz_start = w*stride
                    horiz_end = horiz_start + f
                    a_prev_slice = A[vert_start:vert_end, horiz_start:horiz_end, c]
                    mask = a_prev_slice == np.max(a_prev_slice)
                    dX[vert_start:vert_end, horiz_start:horiz_end, c] += mask * dA[h, w, c]
        return dX
    
    def flatten_backward(self, dA, A):
        (n_H, n_W, n_C) = A.shape
        dA = dA.reshape(n_H, n_W, n_C)
        return dA

    def softmax_backward(self, dA, Z):
        dZ = dA
        return dZ

    def backward_propagation(self, X, y, A4):
        m = y.shape[0]
        dZ4 = A4 - y
        dW4 = 1/m * A3.T.dot(dZ4)
        db4 = 1/m * np.sum(dZ4, axis=0, keepdims=True)
        dA3 = dZ4.dot(self.W4.T)
        dZ3 = self.relu_backward(dA3, Z3)
        dW3 = 1/m * F.T.dot(dZ3)
        db3 = 1/m * np.sum(dZ3, axis=0, keepdims=True)
        dF = dZ3.dot(self.W3.T)
        dP2 = self.flatten_backward(dF, P2)
        dA2 = self.maxpool2d_backward(dP2, A2, 2, 2)
        dZ2 = self.relu_backward(dA2, Z2)
        dP1, dW2, db2 = self.conv2d_backward(dZ2, P1, self.W2, self.b2, 1, 1)
        dA1 = self.maxpool2d_backward(dP1, A1, 2, 2)
        dZ1 = self.relu_backward(dA1, Z1)
        dX, dW1, db1 = self.conv2d_backward(dZ1, X, self.W1, self.b1, 1, 1)
        return dX, dW1, db1, dW2, db2, dW3, db3, dW4, db4
    
    def update_parameters(self, dW1, db1, dW2, db2, dW3, db3, dW4, db4):
        self.W1 -= self.learning_rate * dW1
        self.b1 -= self.learning_rate * db1
        self.W2 -= self.learning_rate * dW2
        self.b2 -= self.learning_rate * db2
        self.W3 -= self.learning_rate * dW3
        self.b3 -= self.learning_rate * db3
        self.W4 -= self.learning_rate * dW4
        self.b4 -= self.learning_rate * db4

    def train(self, X, y, epochs, batch_size):
        for i in range(epochs):
            for j in range(0, X.shape[0], batch_size):
                X_batch = X[j:j+batch_size]
                y_batch = y[j:j+batch_size]
                A4 = self.forward_propagation(X_batch)
                cost = self.compute_cost(A4, y_batch)
                dX, dW1, db1, dW2, db2, dW3, db3, dW4, db4 = self.backward_propagation(X_batch, y_batch, A4)
                self.update_parameters(dW1, db1, dW2, db2, dW3, db3, dW4, db4)
            if i % 10 == 0:
                print("Cost after epoch %i: %f" %(i, cost))
        return self

    def predict(self, X):
        A4 = self.forward_propagation(X)
        return np.argmax(A4, axis=1)
    
    def score(self, X, y):
        y_pred = self.predict(X)
        return np.mean(y_pred == y)
    
    def save(self, filename):
        with open(filename, 'wb') as f:
            pickle.dump(self, f)
        
    @staticmethod
    def load(filename):
        with open(filename, 'rb') as f:
            return pickle.load(f)
        
    def __repr__(self):
        return "ConvolutionalNeuralNetwork(learning_rate={}, epochs={}, batch_size={})".format(self.learning_rate, self.epochs, self.batch_size)



In [3]:
# Train the model

# def __init__(self, X_train, y_train, X_test, y_test, epochs, learning_rate, batch_size):

cnn = CNN(X_train, y_train, X_test, y_test , learning_rate=0.001, epochs=100, batch_size=32)

In [4]:
cnn.train(X_train, y_train, epochs=100, batch_size=32)

(32, 28, 28, 1)


ValueError: too many values to unpack (expected 3)